In [21]:
using NLOptControl
using JuMP
using Ipopt
using Parameters
using Plots
gr()

steps = 20; tests = 5;
global t_solve = Array(Float64, steps, tests) 
global abs_error = Array(Float64, steps, tests) 
global status = Array(Symbol, steps, tests) 
global N = [i+1 for i in 1:steps]
global labels = Array(Symbol,tests)
##################################
# Define NLOptControl problem
##################################
function BrysonDenham{T<:Any}(n::NLOpt,x::Array{T,2},u::Array{T,2}) # dynamic constraint equations
  if n.integrationMethod==:tm
    L = size(x)[1];
  else
    L = size(x)[1]-1;
  end
  dx = Array(Any,L,n.numStates)
  dx[:,1] =  @NLexpression(mdl, [j=1:L], x[j,2] );
  dx[:,2] =  @NLexpression(mdl, [j=1:L], u[j,1] );
  return dx
end
L=1/9;
for q in 1:tests
    for j in 1:steps
        for m in 1:2
            if m==1
                global n = NLOpt();
                n = define(n,stateEquations=BrysonDenham,numStates=2,numControls=1,X0=[0.,1],XF=[0.,-1.],XL=[0.,-Inf],XU=[L,Inf],CL=[-Inf],CU=[Inf])
                if q==1
                    n = configure(n::NLOpt,Ni=1,Nck=[N[j]];(:integrationMethod => :ps),(:integrationScheme => :lgrExplicit),(:finalTimeDV => false),(:tf => 1.0))
                elseif q==2
                    n = configure(n::NLOpt,N=N[j];(:integrationMethod => :tm),(:integrationScheme => :bkwEuler),(:finalTimeDV => false),(:tf => 1.0))
                elseif q==3
                    n = configure(n::NLOpt,N=N[j];(:integrationMethod => :tm),(:integrationScheme => :trapezoidal),(:finalTimeDV => false),(:tf => 1.0))
                elseif q==4 
                    n = configure(n::NLOpt,Ni=2,Nck=[N[j],N[j]];(:integrationMethod => :ps),(:integrationScheme => :lgrExplicit),(:finalTimeDV => false),(:tf => 1.0))
                else
                    n = configure(n::NLOpt,Ni=4,Nck=[N[j],N[j],N[j],N[j]];(:integrationMethod => :ps),(:integrationScheme => :lgrExplicit),(:finalTimeDV => false),(:tf => 1.0))
                end
                ##################################
                # Define JuMP problem
                ##################################
                global mdl = Model(solver = IpoptSolver(print_level=1));
                n,x,u,c=OCPdef(mdl,n)
                global obj = integrate(mdl,n,u[:,1];C=0.5,(:variable=>:control),(:integrand=>:squared))
                @NLobjective(mdl, Min, obj)
                solve(mdl)
            else
                t1 = time();
                status[j,q] = solve(mdl);
                t2 = time();
                t_solve[j,q] = t2 - t1;
                abs_error[j,q]=abs(getvalue(obj) - 4/(9*L))/(4/(9*L))*100;
            end
        end
    end
    if n.integrationMethod==:tm
        labels[q] = string(n.integrationScheme); 
    else
        labels[q] = string(n.integrationScheme, " with Ni = ",n.Ni); 
    end
end

In [23]:
p1=plot(0);
for q in 1:tests
    scatter!(t_solve[:,q],abs_error[:,q],marker = (:star8, 10-q, 0.9),leg=:bottomright,label= string(labels[q]))
end
ylabel!("% error")
xlabel!("solve time (s)")
#ylims!(0, .1)
#xlims!(0, .1)

p2=plot(0);
for q in 1:tests
    scatter!(t_solve[:,q],N,marker = (:star8, 10-q, 0.9),leg=:bottomright,label = string(labels[q]))
end
ylabel!("N")
xlabel!("solve time (s)")
#xlims!(0, .1)

plot(p1,p2,layout=(2,1),background_color_subplot=RGB(0.2,0.2,0.2), background_color_legend=RGB(1,1,1))
plot!(foreground_color_grid=RGB(1,1,1))


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.01 
 
 
 0.02 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 solve time (s) 
 
 
 % error 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 lgrExplicit with Ni = 1 
 
 
 
 
 
 
 bkwEuler 
 
 
 
 
 
 
 trapezoidal 
 
 
 
 
 
 
 lgrExplicit with Ni = 2 
 
 
 
 
 
 
 lgrExplicit with Ni = 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.01 
 
 
 0.02 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 solve time (s) 
 
 
 N 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 lgrExplicit with Ni = 1 
 
 
 
 
 
 
 bkwEuler 
 
 
 
 
 
 
 trapezoidal 
 
 
 
 
 
 
 lgrExplicit with Ni = 2 
 
 
 
 
 
 
 lgrExplicit with Ni = 4

In [24]:
# resize to save
plot!(titlefont=font(14),guidefont=font(14),tickfont=font(12),legendfont=font(12))
#plot!(leg=:best)
plot!(size=(700,850))
savefig("test5a2.png")

In [25]:
p1=plot(0)
for q in 1:tests
    scatter!(N,abs_error[:,q],marker = (:star8, 10-q, 0.9),leg=:bottomright,label = string(labels[q]))
end
xlabel!("N")
ylabel!("% error")

p2=plot(0)
for q in 1:tests
    scatter!(N,abs_error[:,q],marker = (:star8, 10-q, 0.9),leg=:bottomright,label = string(labels[q]))
end
xlabel!("N")
ylabel!("% error")
ylims!(0, 30)

p3=plot(0)
for q in 1:tests
    scatter!(N,abs_error[:,q],marker = (:star8, 10-q, 0.9),leg=:bottomright,label = string(labels[q]))
end
xlabel!("N")
ylabel!("% error")
ylims!(0, .6)

plot(p1,p2,p3,layout=(3,1),background_color_subplot=RGB(0.2,0.2,0.2), background_color_legend=RGB(1,1,1))
plot!(foreground_color_grid=RGB(1,1,1))


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 N 
 
 
 % error 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 lgrExplicit with Ni = 1 
 
 
 
 
 
 
 bkwEuler 
 
 
 
 
 
 
 trapezoidal 
 
 
 
 
 
 
 lgrExplicit with Ni = 2 
 
 
 
 
 
 
 lgrExplicit with Ni = 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 25 
 
 
 30 
 
 
 N 
 
 
 % error 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 lgrExplicit with Ni = 1 
 
 
 
 
 
 
 bkwEuler 
 
 
 
 
 
 
 trapezoidal 
 
 
 
 
 
 
 lgrExplicit with Ni = 2 
 
 
 
 
 
 
 lgrExplicit with Ni = 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 N 
 
 
 % error 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polygon clip-path="url(#clip04)" style="fill:#00002d; fill-opacity:0.9" points="
 170.724,369.191 169.99,366.167 167.33,367.784 168.948,365.125 165.924,364.391 168.948,363.656 167.33,360.997 169.99,362.615 170.724,359.591 171.458,362.615 
 174.118,360.997 172.5,363.656 175.524,364.391 172.5,365.125 174.118,367.784 171.458,366.167 170.724,

In [26]:
# resize to save
plot!(titlefont=font(14),guidefont=font(14),tickfont=font(12),legendfont=font(12))
#plot!(leg=:best)
plot!(size=(700,850))
savefig("test5b2.png")

In [27]:
p1=plot(0);
for q in 1:tests
    scatter!(t_solve[:,q],abs_error[:,q],marker = (:star8, 10-q, 0.9),leg=:bottomright,label= string(labels[q]))
end
ylabel!("% error")
xlabel!("solve time (s)")
ylims!(0, 10)
#xlims!(0, .1)

p2=plot(0);
for q in 1:tests
    scatter!(t_solve[:,q],abs_error[:,q],marker = (:star8, 10-q, 0.9),leg=:bottomright,label= string(labels[q]))
end
ylabel!("% error")
xlabel!("solve time (s)")
ylims!(0, 1)
#xlims!(0, .1)

p3=plot(0);
for q in 1:tests
    scatter!(t_solve[:,q],abs_error[:,q],marker = (:star8, 10-q, 0.9),leg=:bottomright,label= string(labels[q]))
end
ylabel!("% error")
xlabel!("solve time (s)")
ylims!(0, .1)
#xlims!(0, .1)

plot(p1,p2,p3,layout=(3,1),background_color_subplot=RGB(0.2,0.2,0.2), background_color_legend=RGB(1,1,1))
plot!(foreground_color_grid=RGB(1,1,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.01 
 
 
 0.02 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 solve time (s) 
 
 
 % error 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 lgrExplicit with Ni = 1 
 
 
 
 
 
 
 bkwEuler 
 
 
 
 
 
 
 trapezoidal 
 
 
 
 
 
 
 lgrExplicit with Ni = 2 
 
 
 
 
 
 
 lgrExplicit with Ni = 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.01 
 
 
 0.02 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 solve time (s) 
 
 
 % error 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 lgrExplicit with Ni = 1 
 
 
 
 
 
 
 bkwEuler 
 
 
 
 
 
 
 trapezoidal 
 
 
 
 
 
 
 lgrExplicit with Ni = 2 
 
 
 
 
 
 
 lgrExplicit with Ni = 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.01 
 
 
 0.02 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 solve time (s) 
 
 
 % error 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 lgrExplicit with Ni = 1 
 
 
 
 
 
 
 bkwEuler 
 
 
 
 
 
 
 trapezoidal 
 
 
 
 
 
 
 lgrExplicit with Ni = 2 
 
 
 
 
 
 
 lgrExplicit with Ni = 4

In [28]:
# resize to save
plot!(titlefont=font(14),guidefont=font(14),tickfont=font(12),legendfont=font(12))
#plot!(leg=:best)
plot!(size=(700,850))
savefig("test5c2.png")